In [149]:
import sqlite3
import numpy as np
import pandas as pd
conn=sqlite3.connect("factbook.db")

##Playing around with sql db initially to learn connection methods
data=conn.execute("select name from facts order by population asc limit 10;").fetchall()
data


[('Ashmore and Cartier Islands',),
 ('Coral Sea Islands',),
 ('Heard Island and McDonald Islands',),
 ('Clipperton Island',),
 ('French Southern and Antarctic Lands',),
 ('Bouvet Island',),
 ('Jan Mayen',),
 ('British Indian Ocean Territory',),
 ('South Georgia and South Sandwich Islands',),
 ('Navassa Island',)]

In [150]:
###USing pandas to read sql db

dataframe=pd.read_sql("select * from facts ;",conn)

dataframe.head(10)




,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,created_at,updated_at
0,1,af,Afghanistan,652230.0,652230.0,0.0,32564342.0,2.32,38.57,13.89,1.51,2015-11-01 13:19:49.461734,2015-11-01 13:19:49.461734
1,2,al,Albania,28748.0,27398.0,1350.0,3029278.0,0.30,12.92,6.58,3.30,2015-11-01 13:19:54.431082,2015-11-01 13:19:54.431082
2,3,ag,Algeria,2381741.0,2381741.0,0.0,39542166.0,1.84,23.67,4.31,0.92,2015-11-01 13:19:59.961286,2015-11-01 13:19:59.961286
3,4,an,Andorra,468.0,468.0,0.0,85580.0,0.12,8.13,6.96,0.00,2015-11-01 13:20:03.659945,2015-11-01 13:20:03.659945
4,5,ao,Angola,1246700.0,1246700.0,0.0,19625353.0,2.78,38.78,11.49,0.46,2015-11-01 13:20:08.625072,2015-11-01 13:20:08.625072
5,6,ac,Antigua and Barbuda,442.0,442.0,0.0,92436.0,1.24,15.85,5.69,2.21,2015-11-01 13:20:13.049627,2015-11-01 13:20:13.049627
6,7,ar,Argentina,2780400.0,2736690.0,43710.0,43431886.0,0.93,16.64,7.33,0.00,2015-11-01 13:20:18.233063,2015-11-01 13:20:18.233063
7,8,am,Armenia,29743.0,28203.0,1540.0,3056382.0,0.15,13.61,9.34,5.80,2015-11-01 13:20:23.048753,2015-11-01 13:20:23.048753
8,9,as,Australia,7741220.0,7682300.0,58920.0,22751014.0,1.07,12.15,7.14,5.65,2015-11-01 13:20:28.186341,2015-11-01 13:20:28.186341
9,10,au,Austria,83871.0,82445.0,1426.0,8665550.0,0.55,9.41,9.42,5.56,2015-11-01 13:20:33.093597,2015-11-01 13:20:33.093597


In [172]:
dataframe=dataframe.dropna(subset=["area_land","population","population_growth"])

###def a function to apply to the whole dataframe to get population of each city in 2050
def population_2050(df):
    ###form a df with only population and population_growth. filter_out the data with NaNs in both columns
    pop_series=df["population"]
    growth_series=df["population_growth"]
    ### because we have 35 years from 2015 and growth rate is ain %
    growth_multiplier=np.exp(growth_series*0.35)
    population_2050=pop_series*growth_multiplier
    return population_2050


dataframe["population_2050"]=population_2050(dataframe)
dataframe.sort_values("population_2050",inplace=True,ascending=False)
dataframe["name"].head(10)






  
    
    

    



76                                 India
36                                 China
128                              Nigeria
185                        United States
77                             Indonesia
131                             Pakistan
13                            Bangladesh
23                                Brazil
39     Congo, Democratic Republic of the
113                               Mexico
Name: name, dtype: object

In [167]:
### Land Area: Water Area in the world 

land_area=dataframe["area_land"].sum()
Water_area=dataframe["area_water"].sum()
Ratio=land_area/Water_area
Ratio



28.669322660194847

In [168]:
###Which 5 countries have the lowest population density?
dataframe["population_density"]=dataframe["population"]/dataframe["area_land"]
ascending_by_density=dataframe.sort_values("population_density")
print(ascending_by_density["name"].head())



206                            Greenland
223                             Svalbard
231    Falkland Islands (Islas Malvinas)
237                     Pitcairn Islands
117                             Mongolia
Name: name, dtype: object


In [169]:
###Which 5 countries have the highest population density?

ascending_by_density["name"].tail()

203                  Hong Kong
155                  Singapore
116                     Monaco
204                      Macau
189    Holy See (Vatican City)
Name: name, dtype: object

In [170]:
###Which country receives the most immigrants?
dataframe[dataframe["migration_rate"]==dataframe["migration_rate"].max()]["name"]


140    Qatar
Name: name, dtype: object

In [160]:
###Which top country lose the most emigrants?
dataframe[dataframe["migration_rate"]==dataframe["migration_rate"].min()]["name"]





3                                            Andorra
6                                          Argentina
10                                        Azerbaijan
11                                      Bahamas, The
17                                            Belize
18                                             Benin
19                                            Bhutan
26                                      Burkina Faso
28                                           Burundi
33                          Central African Republic
42                                     Cote d'Ivoire
51                                           Ecuador
54                                 Equatorial Guinea
55                                           Eritrea
69                                            Guinea
70                                     Guinea-Bissau
84                                             Japan
90                                      Korea, South
98                                           L